In [1]:
# Jieyu Chen is responsible for this code
# import packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sb
import sklearn
%matplotlib inline

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV

In [5]:
# read csv files, uncomment to switch dataset
# df = pd.read_csv('../dataset/train_with_countries.csv', index_col=0)
df = pd.read_csv('../dataset/train_no_countries.csv', index_col=0)
y_train = df['play_count'].astype(float)
df = df.drop(['play_count'], axis=1)
X_train = df.astype(float)

# df_test = pd.read_csv('../dataset/test_with_countries.csv', index_col=0)
df_test = pd.read_csv('../dataset/test_no_countries.csv', index_col=0)
y_val = df_test['play_count'].astype(float)
df_test = df_test.drop(['play_count'], axis=1)
X_val = df_test.astype(float)
df_test.head()

,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,KEY,duration_ms,...,valence,mode,artist_followers,artist_popularity,disc_number,explicit,album_release_date,album_total_tracks,album_available_markets,listener_count
0,0.549,0.190,107.974,0.0252,0.6130,0.000131,4.0,0.602,10.0,280066,...,0.518,1.0,38974.0,49.0,1,0,274,1,74,4019.0
1,0.627,0.123,129.902,0.0355,0.0977,0.000002,3.0,0.405,8.0,204734,...,0.334,1.0,300056.0,57.0,1,0,274,21,77,1.0
2,0.193,0.113,89.154,0.0421,0.8620,0.000010,4.0,0.490,11.0,189506,...,0.184,1.0,1019811.0,67.0,1,0,274,100,77,678401.0
3,0.759,0.350,150.032,0.1210,0.2570,0.000065,4.0,0.544,10.0,184320,...,0.303,1.0,408712.0,69.0,1,0,274,12,78,79546.0
4,0.344,0.152,131.731,0.0470,0.7720,0.359000,4.0,0.305,5.0,186773,...,0.393,1.0,663.0,28.0,1,0,274,23,79,31.0


In [7]:
# MinMax Scaling
transformer = MinMaxScaler().fit(X_train) 
X_train = transformer.transform(X_train)
X_train = pd.DataFrame(X_train)

transformer = MinMaxScaler().fit(X_val) 
X_val = transformer.transform(X_val)
X_val = pd.DataFrame(X_val)

In [12]:
# Linear SVR
# run the SVM
svr = LinearSVR()
svr.fit(X_train, y_train)

# predict
y_train_pred = svr.predict(X_train)
y_val_pred = svr.predict(X_val)

# random search parameters
parameters = {'epsilon': [0.0, 0.2], 
              'tol': [1e-3, 1e-4, 1e-5, 1e-6, 1e-7], 
              'C': [0.6, 0.3, 0.2], 
              'dual': [False],
              'loss': ['squared_epsilon_insensitive'],
              'max_iter': [7000, 15000],
              'fit_intercept': [False]
             }

# perform random search
random_search = RandomizedSearchCV(svr, parameters, cv=5)
random_search.fit(X_train, y_train)

# random search predict
y_train_pred = random_search.predict(X_train)
y_val_pred = random_search.predict(X_val)

# print best parameters
print(random_search.best_estimator_)

# print errors
print("train MAE: ", mean_absolute_error(y_train, y_train_pred))
print("test MAE: ", mean_absolute_error(y_val, y_val_pred))

LinearSVR(C=0.2, dual=False, epsilon=0.2, fit_intercept=False,
          intercept_scaling=1.0, loss='squared_epsilon_insensitive',
          max_iter=15000, random_state=None, tol=0.001, verbose=0)
train MAE:  39116.46570809588
test MAE:  89716.39857503022
